# Machine learning from woods - exploring tree-based ensemble models

### Intro

### ML Background

#### Decision Trees

#### Random Forest

#### AdaBoost

#### Gradient Tree Boosting

### Ensemble models in scikit-learn

### Datasets

### Results

### Model tuning

#### Hyperparameters

#### Cross-validation

### Summary